In [ ]:
import datetime
import json

import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon
from shapely import affinity
import matplotlib.pyplot as plt
import contextily as ctx
from netCDF4 import Dataset

from bokeh.io import output_file, output_notebook, show
from bokeh.models import (
  GMapPlot, GMapOptions, ColumnDataSource, Circle, LogColorMapper, BasicTicker, ColorBar,
    DataRange1d, Range1d, PanTool, WheelZoomTool, BoxSelectTool, ResetTool, HoverTool
)
from bokeh.models.mappers import ColorMapper, LinearColorMapper
from bokeh.palettes import Viridis256
from bokeh.plotting import figure
from bokeh.resources import CDN
from bokeh.embed import file_html

In [ ]:
with open('../keys.json', 'r') as f:
    keys = json.loads(f.read())

## get lake shape file

In [ ]:
gdf = gpd.read_file('../data/na_lakes/hydrography_p_lakes_v2.shp')

In [ ]:
gdf = gdf.to_crs(epsg=4326)
gdf = gdf.dropna()

In [ ]:
gdf[gdf['NAMEEN'].str.contains('Seneca')]

In [ ]:
lake_bounds = gdf[gdf['NAMEEN'].str.contains('Seneca')]['geometry'].unary_union

## get sst metadata

In [ ]:
sst_metadata = Dataset("../data/sst_nowcoast/202006/20200601_0600_sport_nhemis_sstcomp_2km_unscaled.nc", "r", format="NETCDF4")
print(sst_metadata.data_model)

In [ ]:
sst_metadata.variables.keys()

In [ ]:
plt.imshow(sst_metadata.variables['Band1'][:].mask, cmap='viridis', interpolation='nearest')
plt.colorbar()
plt.show()

In [ ]:
plt.imshow(sst_metadata.variables['Band1'][:].data, cmap='viridis', interpolation='nearest')
plt.colorbar()
plt.figure(figsize=(18, 18))
plt.show()

In [ ]:
water = sst_metadata.variables['Band1'][:].data

In [ ]:
less_water = water[1300:2800,2800:6100]

In [ ]:
less_water.shape

In [ ]:
plt.imshow(less_water, cmap='viridis', interpolation='nearest')
plt.colorbar()
plt.figure(figsize=(18, 18))
plt.show()

In [ ]:
water_mask = sst_metadata.variables['Band1'][:].mask
water_points = []
for i in range(1300, 2800):
    for j in range(2800, 6100):
        if water_mask[i][j] == 0:
            water_points.append( [i, j] )

water_points[:5]

In [ ]:
plt.imshow(sst_metadata.variables['Band1'][:].data, cmap='viridis', interpolation='nearest')
plt.colorbar()
plt.figure(figsize=(18, 18))
plt.show()

In [ ]:
sst_data = Dataset("../data/sst_nowcoast_jun2020/20200601_0600_sport_nhemis_sstcomp_2km_scaled.nc", "r", format="NETCDF4")

In [ ]:
lats = sst_data.variables['lat'][:].data
lons = sst_data.variables['lon'][:].data
temps = sst_data.variables['Band1'][:].data

In [ ]:
water_temps = [ temps[tuple(point)] for point in water_points ]
len(water_temps)

In [ ]:
coords = []
for idx in water_points:
    i, j = idx
    coords.append(Point((lons[j], lats[i])))
len(coords)

In [ ]:
# load lake points as geodataframe
df = pd.DataFrame(data={ 'idx': water_points, 'temp': water_temps})

gdf = gpd.GeoDataFrame(df, crs='epsg:4326', geometry=coords)
gdf.head()

In [ ]:
map_options = GMapOptions(lat=42.6681422, lng=-73.8457002, map_type="roadmap", zoom=6)

plot = GMapPlot(
    x_range=Range1d(), y_range=Range1d(), map_options=map_options
)
plot.title.text = "Lake Points with Temp Gradient"

plot.api_key = keys['google_maps_api_key']
source = ColumnDataSource(
    data=dict(
        lat=gdf['geometry'].y,
        lon=gdf['geometry'].x,
        size=[10]*len(gdf),
        temp=gdf['temp']
    )
)

color_mapper = LinearColorMapper(palette=Viridis256)

circle = Circle(
    x="lon", 
    y="lat", 
    size="size", 
    fill_color={'field': 'temp', 'transform': color_mapper}, 
    fill_alpha=1, 
    line_color='black'
)
plot.add_glyph(source, circle)

color_bar = ColorBar(
    color_mapper=color_mapper, 
    ticker=BasicTicker(), 
    label_standoff=12, 
    border_line_color=None, 
    location=(0,0)
)
plot.add_layout(color_bar, 'right')


plot.add_tools(PanTool(), WheelZoomTool(), BoxSelectTool(), ResetTool(), HoverTool(tooltips=[('temp', '@temp')]))

output_notebook()
show(plot)

In [ ]:
gdf = gdf.assign(**{ 'lake': gdf.within(lake_bounds) })
lake_data = gdf[gdf['lake']]
lake_data.head()

In [ ]:
len(lake_data)

In [ ]:
map_options = GMapOptions(lat=42.6681422, lng=-73.8457002, map_type="roadmap", zoom=6)

plot = GMapPlot(
    x_range=Range1d(), y_range=Range1d(), map_options=map_options
)
plot.title.text = "Lake Points with Temp Gradient"

plot.api_key = keys['google_maps_api_key']
source = ColumnDataSource(
    data=dict(
        lat=lake_data['geometry'].y,
        lon=lake_data['geometry'].x,
        size=[10]*len(lake_data),
        temp=lake_data['temp']
    )
)

color_mapper = LinearColorMapper(palette=Viridis256)

circle = Circle(
    x="lon", 
    y="lat", 
    size="size", 
    fill_color={'field': 'temp', 'transform': color_mapper}, 
    fill_alpha=1, 
    line_color='black'
)
plot.add_glyph(source, circle)

color_bar = ColorBar(
    color_mapper=color_mapper, 
    ticker=BasicTicker(), 
    label_standoff=12, 
    border_line_color=None, 
    location=(0,0)
)
plot.add_layout(color_bar, 'right')


plot.add_tools(PanTool(), WheelZoomTool(), BoxSelectTool(), ResetTool(), HoverTool(tooltips=[('temp', '@temp')]))

output_notebook()
show(plot)

In [ ]:
all_idxs = []
all_coords = []
all_temps = []
all_dates = []
for d in range(1, 23):
    for cycle in ['0600', '1800']:
        print(f"202006{d:02}_{cycle}")
        sst_data = Dataset(f"../data/sst_nowcoast_jun2020/202006{d:02}_{cycle}_sport_nhemis_sstcomp_2km_scaled.nc", "r", format="NETCDF4")
        
        for idx in list(lake_data['idx']):
            
            all_idxs.append(str(idx))
            all_dates.append(datetime.datetime(2020, 6, d, int(cycle[:2]), 0))

            all_coords.append(Point((sst_data.variables['lon'][:].data[tuple(idx)[1]], sst_data.variables['lat'][:].data[tuple(idx)[0]])))

            all_temps.append(sst_data.variables['Band1'][:].data[tuple(idx)])
        

In [ ]:
seneca_lake_points = pd.DataFrame(data={ 
    'idx': all_idxs,
    'date': all_dates,
    'temp': all_temps
})

seneca_lake_points = gpd.GeoDataFrame(seneca_lake_points, crs='epsg:4326', geometry=all_coords)
seneca_lake_points.head()

In [ ]:
seneca_lake_points['temp'].unique()[7]

In [ ]:
seneca_lake_points['temp'] = seneca_lake_points['temp'].replace([seneca_lake_points['temp'].unique()[7],], np.nan)

In [ ]:
import plotly.express as px

In [ ]:
fig = px.line(
    seneca_lake_points,
    x='date',
    y='temp',
    color='idx',
    width=1920, 
    height=1080
)
fig.update_layout(
    title="HRRRX/CLM Lake Model 0hr-Preds for Seneca Lake, NY from Apr 20, 2020 - Jun 20, 2020 Forecast Cycles 00, 06, 12, and 18",
    xaxis_title="Forcast Cycle",
    yaxis_title="Water Temp (C)",
    xaxis = {
        'dtick': 3600000.0*24#*7
    },
    yaxis = {
        'range': [0, 80]
    }
)
fig.update_traces(connectgaps=False)
fig.show()